# Causal inference applied to an electric circuit
This will be a case of causal inference applied to a simple electric circuit. The primary function of this circuit will be defined by a specific causal relationship, i.e. the position of the switch should *cause* the light to be on or off. Failure modes will *cause* a specific perturbation in the primary function. Statistical associations are generally insufficient to identify causalities. A Directed Acyclic Graph (DAG) will be introduced as a representation of the expert knowledge that should be presumed to identify a causality by a statistical association. This case will *apply* machine learning to detect faults using causal inference. The practical challenges rather than the theoretical foundations will be important here.

This case will firstly introduce the circuit. It will proceed by inferring a causality between the switch and the light. Then, it will introduce a single failure mode that *causes* a perturbation in the primary function. Finally, the case will be generalised to multiple failure modes. The script will revisit the case using extended, realistic time series using a random forest algorithm and k-means clustering.


## Introduction to the electric circuit

<table>
    <thead>
        <tr>
            <th> <th>Time <th>V<sub>0</sub> <th> V<sub>1</sub> <th> S<sub>1</sub>
        </tr>
    </thead>
    <tbody>
        <tr>
            <td rowspan=11> <img src="figures/CausalInference01.png" width="350"/>  <td>[  0:  4] <td> 0,0 <td> 0,0 <td> 0
        </tr>
        <tr>
            <td>[  5:14] <td> 3,4 <td> 1,5 <td> 1
        </tr>
        <tr>
            <td>[15:18] <td> 0,0 <td> 0,0 <td> 0
        </tr>
        <tr>
            <td>[19:23] <td> 3,4 <td> 1,5 <td> 1
        </tr>
        <tr>
            <td>[24:30] <td> 0,0 <td> 0,1 <td> 0
        </tr>
        <tr>
            <td>[31:36] <td> 3,5 <td> 1,4 <td> 1
        </tr>
        <tr>
            <td>[37:38] <td> 0,0 <td> 0,0 <td> 0
        </tr>
        <tr>
            <td>[39:40] <td> 0,0 <td> 0,0 <td> 0
        </tr>
        <tr>
            <td>[41:49] <td> 2,5 <td> 2,5 <td> 1
        </tr>
        <tr>
            <td>[50:53] <td> 3,5 <td> 1,5 <td> 1
        </tr>
        <tr>
            <td>[54:60] <td> 0,0 <td> 0,0 <td> 0
        </tr>
    </tbody>    
</table>



The electric circuit in Figure 1 consists of a light, two resistors, a ground connection and a switch. From this circuit, the following measurements have been recorded: 
- the position of the switch ($S_{1}$), 
- the voltage $V_{0}$ over one resistor and the light,
- the voltage $V_{1}$ over one resistor.

## Challenge to assign health labels

As the circuit is healthy, the voltages $V_{0}=0V$ and $V_{1}=0V$ at an open switch ($S_{1}=0$) and the voltages $V_{0}=3,4V$ and $V_{1}=1,4V$ at a closed switch ($S_{1}=1$). As the circuit is faulty, the voltages may take different values. Ideally, the health state of the circuit is entirely identified by the measurements of $S_{1}$, $V_{0}$, and $V_{1}$. Figure 2 shows that the health state of the circuit is *not* determined by the measurements of $S_{1}$, $V_{0}$, and $V_{1}$ because at an open switch $S_{1}=0$ and $V_{0}=0$, and $V_{1}=0$, the circuit may be healthy, but it may also be faulty. Then, the assignment of a health label requires additional knowledge. Generally, the assignment of health labels is challenging.

Figure 2 shows two scenario's to assign health labels. Figure 2-Left shows a scenario where an expert determined the expected measurements of $S_{1}$, $V_{0}$, and $V_{1}$ at some specific health state. This scenario typically yields training data that are:
- *complete* in the sense that each health state that is deemed possible, is in the training data;
- *balanced* in the sense that each health state that is deemed possible, only occurs once in the training data.

Figure 2-Right shows a scenario where an expert assigned health labels to measurements from the normal course of operations (nonexperimental research). This scenario typically yields training data that are:
- *incomplete* in the sense that not all health states that are deemed possible, are in the training data (fortunately to society);
- *unbalanced* in the sense that not all health labels are evenly represented in the training data.

So, Figure 2-Left follows from expected measurements that have been assigned to possible health states and Figure 2-Right follows from health labels that have been assigned to measurements that occurred. The demo script will show that the composition of the training data determines the behaviour of the machine learning algorithm. Each scenario has pros and cons and there is no generic optimal composition of a training data set.

![image.png](figures/CausalInference02a.png) ![image.png](figures/CausalInference02b.png)

## Causal effect, given a healthy circuit

This circuit is an artefact that should exhibit a specific causal effect. In particular, the light should *listen* to the switch $S_{1}$. Therefore, a causality $S_{1} \rightarrow "Light"$ has been designed in. Although the measurements do not cover the state of the light directly, the voltages $V_{0}$ and $V_{1}$ are indicators for the proper functioning of the circuit, i.e. the occurring of the intended causal effect. Figure 3 shows an arbitrary causal structure that represents the functioning of the circuit.

![image.png](figures/CausalInference03.png)

The switch $S_{1}$ causes a current to flow through the light because the operation of the switch $S_{1}$ changes the current, but a change in the current does not alter the position of the switch $S_{1}$. So, a causal relation is directional. The current is not directly measured, but the voltages $V_{0}$ and $V_{1}$ have been represented as redundant indicators. Redundant because the voltages and the current are just deemed to be related by an equivalence relation (Ohm's law). The burning of the light is presumed to be a redundant indicator of the current. The burning of the light is just observed by the user and it is *not* measured.

As the burning of the light and the voltages are all redundant indicators of the current, the causalities $S_{1} \rightarrow V_{0}$ , $S_{1} \rightarrow V_{1}$, and $S_{1} \rightarrow "Light"$ equally indicate the health of the circuit.

## Causal effect, given a faulty circuit

The user deems the circuit to be failed as the light does not respond to switch $S_{1}$ in a predetermined way. Figure 3 implies that the switch $S_{1}$ and the health of the circuit are both causes of the current. The scatters in Figure 2 only show two data points that indicate a *healthy* circuit, but many data points indicating a faulty circuit. An elementary diagnostic system just checks whether the voltages $V_{0}$ and $V_{1}$ correspond with the switch position.

The presumed causalities in Figure 3 are important to decide about the health using the measurements $S_{1}$, $V_{0}$, and $V_{1}$. For example, if only the position of the switch $S_{1}$ were known, nothing is implied about the health of the circuit. However, if (one of the indicators of) the current is known, one's belief in the health of the circuit may change wildly as Figure 2 shows quite some measurements that unambiguously indicate an unhealthy circuit. So, knowledge of the indicators of the current is more valuable for diagnostics than knowledge of the switch $S_{1}$. It is the presumed causal structure in Figure 3 that collides on the current that explains this difference in diagnostic capability. An explicit causal model could be very helpful to specify an expert's causal assumptions that underlie his diagnostics of the circuit. A veteran engineer is supposed to construct a more adequate causal model than a novice. When given the same measurements, the veteran is supposed to arrive at better conclusions.

Using several indicators for the current could make diagnostics robust against calibration issues of the measurements. By applying an ensemble of machine learning models that only consider some random selection of the measured indicators, the machine learning models that considered that corrupt measurement may be voted away by the others. The random forest model in the demo script is such an ensemble model.

## Elementary diagnostics by identifying a conditional treatment effect

Let's concentrate on the directed acyclic graph (DAG) elements in Figure 3: $S_{1} \rightarrow "current" \leftarrow "health"$.
The collider in this graph could be replaced by any redundant indicator so that: $S_{1} \rightarrow "V_{x}" \leftarrow "health"$. Now, the causal estimand that indicates the primary function of this circuit is given by:

$Pr(V_{x}|do(S_{1}), "health")$

By Bayesian network factorisation, this conditional causal estimand is identifiable by the statistical estimand:

$Pr(V_{x},S_{1},"health")=Pr("health")Pr(S_{1})Pr(V_{x}|"health",S_{1})$

Let the health of the circuit be good. Then, the conditional average treatment effect (ATE) is given by:

$E[V_{x}|S_{1}=closed, "health"= good] - E[V_{x}|S_{1}=openend, health"= good]$

Figure 2 already showed that the voltages at a closed and at an open switch reduce to a single value so that:

$E[V_{0}|S_{1}=closed, "health"= good] - E[V_{0}|S_{1}=openend, health"= good]=3,4-0=3,4V$
$E[V_{1}|S_{1}=closed, "health"= good] - E[V_{1}|S_{1}=openend, health"= good]=1,4-0=1,4V$

For the various faulty states of the circuit, a similar conditional treatment effect could be calculated. Depending on the probability of each of these health states, an unconditional average treatment effect $Pr(V_{x}|do(S_{1}))$ could be estimated. However, the *conditional* average treatment effects are much more important for the purpose of diagnostics. Simply because differences in the *conditional* average treatment effect are informative about the health of the circuit.

## Advanced diagnostics by identifying a conditional treatment effect

Figure 2 already showed that a faulty circuit may exhibit a wide variety of health indicators. In particular, Figure 2-Left relied on some FMEA of possible failure modes. Here, it is presumed that an expert deemed the following failure modes possible:

<table>
    <tr>
        <th>Component <th>Failure mode <th>Code <th>Failure effect
    </tr>
    <tr>
        <td>wire to light <td>break circuit <td> $F_{1}$ <td>No light when switched on
    </tr>
    <tr>
        <td>light <td>break circuit <br /> short circuit <td>$F_{2}$ <br />$F_{3}$ <td>No light when switched on
    </tr>
    <tr>
        <td>wire to ground <td>break circuit <td>$F_{4}$ <td>No light when switched on
    </tr>
</table>

Using the FMEA, the health labels from Figure 2 could be made more precise:

![image.png](figures/CausalInference02c.png) ![image.png](figures/CausalInference02d.png)

Figure 4-Left shows each of the 12 possible health states of the circuit and their *expected* corresponding measurements of $V_{0}$ and $V_{1}$ at each switch position. The number of health states is 12 rather than 16 because a short circuit $F_{2}$ and a break circuit $F_{3}$ cannot occur simultaneously. Figure 4-Right shows some measurements from the circuits course of operations and their assigned health labels. As faults are typically exceptional, many of the possible fault states remain unobserved.

As compared to Figure 2, Figure 4 is much more precise about the health of the circuit. Still, Figure 4-Left shows that several some health states map to the same measurements of $V_{0}$, $V_{1}$ and $S_{1}$. In particular, the values $V_{0}=0V$, $V_{1}=0V$ and $S_{1}=0$ map to many health states. So, albeit the values of $V_{0}$, $V_{1}$ and $S_{1}$ are informative about circuit health, they are insufficient assess the circuit health unconditionally. Extending the diagnostic capability may follow from:
- Feature extension, i.e. generate indicators of spread, Fourier amplitudes,... from the same measurements;
- Measurement extension, i.e. extend the sensor suite.

Figure 4-Right shows that some of the health states that were deemed possible in Figure 4-Left, did not occur during some operations. Generally, Figure 4-Right is informative about the probabilities of the various health states and therefore about reliability or availability. In this specific case, some values in Figure 4-Right did *not* occur in Figure 4-Left. Possibly:
- the circuit shows failure modes $F_{5},...$ that are missing in the FMEA;
- the circuit behaves different than *expected*.

This introduction presented a dichotomous quality assessment in Figure 2, and a multiple state quality assessment in Figure 4. It showed that the health labelling is challenging; in this case of a circuit at an open switch $S_{1}=0$ in particular. Moreover, it showed two scenarios to compose labelled training data for a machine learning algorithm. The "balanced and complete" training data equally represents each health state that is deemed possible. The "unbalanced and incomplete" training data weighs the health states by their occurrence.

Moreover, it should be realised that this case did not involve any direct recordings of the functionality of the circuit, i.e. this circuit cam into being to control light. The health labels in this case revealed that the circuit can be faulty while its functionality corresponds with user intends. So, this case involved *hidden failures*.


# [Click here to see the script](https://nbviewer.jupyter.org/github/chrisrijsdijk/RAMS/blob/master/notebook/Arduino_3Vars/Arduino_diagnostics_ensemble3VarsVal.ipynb) 